In [1]:
# Standard Libraries
import pandas as pd
import numpy as np
import os
import sys
from datetime import date

# Machine Learning
#import tensorflow as tf

# Image Libraries
import cv2
from cv2 import imread, imshow, waitKey, destroyAllWindows, rectangle, CascadeClassifier

# Database
import psycopg2

# Visualizations
from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Get the classifier .xml files and save them in the src directory
! curl -o ../src/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
! curl -o ../src/haarcascade_smile.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_smile.xml
! curl -o ../src/haarcascade_eye.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml
! curl -o ../src/haarcascade_eye_tree_eyeglasses.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye_tree_eyeglasses.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  908k  100  908k    0     0  1697k      0 --:--:-- --:--:-- --:--:-- 1694k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  184k  100  184k    0     0   607k      0 --:--:-- --:--:-- --:--:--     0:-- --:--:--  607k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  333k  100  333k    0     0   989k      0 --:--:-- --:--:-- --:--:--  986k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  587k  100  587k    0     0  1280k      0 --:--:-- --:--:-- --:--:-- 1280k


# Database

In [3]:
def test_database(db='kennedy', user_id='postgres', passwd=''):
    
    con = None

    try:
    
        con = psycopg2.connect(database=db, user=user_id,
            password=passwd)

        cur = con.cursor()
        cur.execute('SELECT version()')

        version = cur.fetchone()[0]
        print(version)

    except psycopg2.DatabaseError as e:

        print('Error {}'.format(e))
        sys.exit(1)

    finally:

        if con:
            con.close()
            
    return

In [4]:
def connect_database(db='kennedy', user_id='postgres', passwd=''):
    
    con = None

    try:
    
        con = psycopg2.connect(database=db, user=user_id,
            password=passwd)
        cur = con.cursor()
        
    except psycopg2.DatabaseError as e:
        
        print('Error {}'.format(e))
        sys.exit(1)
        
    return con, cur

In [5]:
def close_database(con):
    
    if con:
        con.commit()
        con.close()
        
    return 1

In [6]:
def create_database(con, cur):

        with con:

            cur.execute("DROP TABLE IF EXISTS images")
            cur.execute("CREATE TABLE images(id BIGINT DEFAULT 0 NOT NULL, file_name TEXT, dir_path TEXT, date DATE DEFAULT '1/1/1970' NOT NULL, img_orig BYTEA, face BOOLEAN DEFAULT 'False' NOT NULL, img_proc BYTEA, face_class INTEGER DEFAULT 0 NOT NULL)")
            cur.execute("COMMENT ON COLUMN images.id IS E'This number will come from a counter programatically.'")
            cur.execute("COMMENT ON COLUMN images.dir_path IS E'directory path to original image.  This is so that you can trace back to the imported data for files with the same names in multiple directories.'")
            cur.execute("COMMENT ON COLUMN images.date IS E'date image was imported.'")
            cur.execute("COMMENT ON COLUMN images.face_class IS E'Classes based off _# in filename.  0,1,2,3,4,5. 0=no_class, 1=commecicomm,2=smile,3=shut,4=shocked,5=sunglasses.'")
            cur.execute("COMMENT ON COLUMN images.img_proc IS E'Processed images!'")
            cur.execute("ALTER TABLE images ADD CONSTRAINT dir_path PRIMARY KEY(id)")
            cur.execute("ALTER TABLE images ADD COLUMN eyes BOOLEAN DEFAULT 'False' NOT NULL")
            cur.execute("ALTER TABLE images ADD COLUMN mouth BOOLEAN DEFAULT 'False' NOT NULL")
            cur.execute("ALTER TABLE images ADD COLUMN nose BOOLEAN DEFAULT 'False' NOT NULL")
            cur.execute("ALTER TABLE images ADD COLUMN class_pred INTEGER DEFAULT 0 NOT NULL")
            cur.execute("ALTER TABLE images ADD COLUMN ext TEXT DEFAULT '.jpg' NOT NULL")
            con.commit()
        return 1
        

In [7]:
def bulk_insert_database(con, cur, id_, file_name, dir_path):

    with con:
        
        today = date.today().strftime('%m/%d/%Y')
        imgopen = open(dir_path, 'rb').read()
        face_class = file_name[-5]
        data = (id_, dir_path, today, psycopg2.Binary(imgopen) , False, face_class)
        query = "INSERT INTO images(id, dir_path, date, img_orig, face, face_class) VALUES (%s, %s, %s, %s, %s, %s)"
        cur.execute(query, data)
        
    return con, cur

In [8]:
def load_images_database(fname):
    i = 0
    for file in fname:
        file_name = os.path.basename(file)
        file_path = file
        print('{}...{}...{}'.format(i, file_name, file_path))
        bulk_insert_database(con, cur, i, file_name, file_path) 
        i+=1
    return 1

In [9]:
# Update to take id and path as inputs.  id to select a specific row, path to where you wish to output (visualizations)
def write_image_database_to_file(con, cur):
    try:
        
        data = ('id', 'dir_path', 'date', 'img_orig' , 'face', 'face_class')
        cur.execute("SELECT data FROM images")
        data = cur.fetchone()
        open(os.path.expanduser('~/Desktop/test.jpp'), 'wb').write(data[3])        

    except psycopg2.DatabaseError as e:

        print('Error {}'.format(e))
        sys.exit(9)


In [24]:
def read_image_database(con):
    query = "SELECT id, dir_path, img_orig, face, face_class, class_pred FROM images"
    df_table = pd.read_sql_query(query, con)
    return df_table

In [12]:
def write_image_database(con, cur, data):
#     data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
#     cv2.imwrite('../test/test_image.jpg', data)
    return  7

# Preprocessing

In [13]:
file_name = '' # image name
file_path = '' # path to file

In [14]:
# Process Images Directory for Feeding into the Model
def build_image_list(path):
    '''
    Search (path) and make a recursive listing of paths to each file.
    INPUTS: path = top of directory tree to begin recursion for images. ex) "../data"
    OUTPUTS: fname = A full path listing of all files in the directory tree.  astype.list()
    '''
    fname = []
    for root,d_names,f_names in os.walk(path):
        f_names = [f for f in f_names if not f[0] == '.'] # skip hidden files: .DSstore
        d_names = [d for d in d_names if not d[0] == '.'] # skip hidden folders: .git
        for f in f_names:
            fname.append(os.path.join(root, f))
    return fname

In [19]:
image_list = build_image_list('../data/TD_RGB_E')
print(len(image_list))
print(image_list[0])
#print(image_list)

560
../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_5.jpg


In [20]:
test_database()

PostgreSQL 12.1 on x86_64-apple-darwin16.7.0, compiled by Apple LLVM version 8.1.0 (clang-802.0.42), 64-bit


In [21]:
con, cur = connect_database()

In [22]:
create_database(con, cur)
#cur.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'images'")

1

In [23]:
load_images_database(image_list)

0...TD_RGB_E_5.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_5.jpg
1...TD_RGB_E_4.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_4.jpg
2...TD_RGB_E_1.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_1.jpg
3...TD_RGB_E_3.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_3.jpg
4...TD_RGB_E_2.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_2.jpg
5...TD_RGB_E_5.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/92/TD_RGB_E_5.jpg
6...TD_RGB_E_4.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/92/TD_RGB_E_4.jpg
7...TD_RGB_E_1.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/92/TD_RGB_E_1.jpg
8...TD_RGB_E_3.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/92/TD_RGB_E_3.jpg
9...TD_RGB_E_2.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/92/TD_RGB_E_2.jpg
10...TD_RGB_E_5.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/104/TD_RGB_E_5.jpg
11...TD_RGB_E_4.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/104/TD_RGB_E_4.jpg
12...TD_RGB_E_1.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/104/TD_RGB_E_1.jpg
13...TD_RGB_E_3.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/104/TD_RGB_E_3.jpg
14...TD_RGB_

1

In [25]:
# read from database
df = read_image_database(con)
df.head()

,id,dir_path,img_orig,face,face_class,class_pred
0,0,../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_5.jpg,"[b'\xff', b'\xd8', b'\xff', b'\xe1', b'\xff', ...",False,5,0
1,1,../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_4.jpg,"[b'\xff', b'\xd8', b'\xff', b'\xe1', b'\xff', ...",False,4,0
2,2,../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_1.jpg,"[b'\xff', b'\xd8', b'\xff', b'\xe1', b'\xff', ...",False,1,0
3,3,../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_3.jpg,"[b'\xff', b'\xd8', b'\xff', b'\xe1', b'\xff', ...",False,3,0
4,4,../data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_2.jpg,"[b'\xff', b'\xd8', b'\xff', b'\xe1', b'\xff', ...",False,2,0


# TTS

In [ ]:
columns=['face_class']
y = df.face_class
X = df.drop(columns, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
print('X_train: {}'.format(len(X_train)))
print('y_train: {}'.format(len(y_train)))
print('X_test: {}'.format(len(X_test)))
print('y_test: {}'.format(len(y_test)))

# Crappy Model

In [ ]:
def detect_face(img_rgb, classifier):
    '''
    1st classifier: HAAR classifier for face detection
    read image from img_orig column
    process face
    if faces detected, update face column from False to True
    draw box around face
    write modified image to img_proc column
    call detect_eyes
    if no face detected, exit loop
    '''
    classifier_full = CascadeClassifier(classifier)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    
    bounding_boxes = classifier_full.detectMultiScale(img_gray, scaleFactor=1.3, minNeighbors=7, minSize=(200, 200))
    box_list = []
    for box in bounding_boxes:
        box_list.append(box)
    if len(box_list) < 1:
        return 0
    else:
        # update face to True
        draw_boxes(img_rgb, box_list)
        img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
        # write img to img_proc
    return img_rgb, roi

## classify emotion

In [ ]:
def predict_emotion(classifier, id_, con):
    '''
    HAAR classifier for face detection. iI faces detected: update feature from False to True, draw box around face, write image to new field in database
    read image from original column
    INPUTS: classifier, image file from database, database column to write. emotions = [ 0,1,2,3,4,5] # 0=no_class, 1=commecicomm,2=smile,3=shut,4=shocked,5=sunglasses
    OUTPUTS: Annotated image saved to database, database flag updated.
    '''
    classifier_fishface = cv2.createFisherFaceRecognizer() #Initialize fisher face classifier
    image = cv2.imread(imagePath) # SELECT img_orig from images WHERE id == id_
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=7, minSize=(50, 50))
    print('Found {} faces!'.format(len(faces)))
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 3)
    # write img to database
    # update flagired
    return

# Annotate Images

In [ ]:
def draw_boxes(img_rgb, box_list):
    '''
    draw boxes detected in box list.  Also calculate roi to be passed to another function.
    INPUTS: img_rgb : image to be modified. 
            box_list : list of box coordinates.
    OUTPUTS: annotated image, roi
    '''
    for box in box_list:
        x, y, w, h = box
        x2, y2 = x + w, y + h
        cv2.rectangle(img_rgb, (x, y), (x2, y2), (0,255,0), 3)
        roi = img(y:y2, x:x2)
    return save_image(img_rgb, roi)

In [ ]:
def save_image(img_rgb, roi):
#     plt.imshow(img)
#     plt.xticks([]), plt.yticks([])
#     plt.show()
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    # write to db cv2.imwrite('../sort/face/'+file_name+'.jpg', img_rgb)
    return

In [ ]:
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    # write to db cv2.imwrite('../sort/face/'+file_name+'.jpg', img_rgb)

In [ ]:
X_test.head()

# Main

In [ ]:
# set classifier
classifier = '../src/haarcascade_frontalface_default.xml'
# read the image
img_rgb = X_train['img_orig']

# detect face - with HAAR cascade
img_rgb, roi = detect_face(img_rgb, classifier)


# scale down


# return roi

# call model for image - call fisherfaces

# write processed image to dataframe

# update flag in dataframe

# choose next image

# once complete, pueh changes to database


# Cleanup

In [ ]:
close_database(con)

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test, y_pred_class)

tn = cm_2[0,0]
fn = cm_2[0,1]
fp = cm_2[1,0]
tp = cm_2[1,1]

accurracy = (tp + tn)/(tn+tp+fn+fp)

precision = tp / (tp+fp)

recall = tp / (tp + fn)
f1_score = 2*precision*recall/(precision+recall)
print('My model metrics were: Accurracy: {}, Precision: {}, Recall: {}, and F1: {}'.format(accurracy,precision,recall,f1_score))


plt.matshow(cm)

plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show();



# MVP

# Add webcam support

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret, img = cap.read()
    '''
    lots of code here
    
    '''
    cv2.imshow('img', img)
    #Display camera feed until ESC key is pressed
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()